# Probe Detection Using Deep Learning
---

### Outline
- System Selection
- 
- Model Testing
    - Train 
    - 
- Inference with Custom Model
- Evaluation
- Future Improvements

The results of this code are accompanied in the following [Report](REport).

## System Selection and Setup


The selection test models are **YOLOv8, RT-DETR, YOLO-NAS ** based on:
- State of Art (speed and acccuracy)
- Reasonable Computational Requirements
- Edge Deployment Scnearios (Jetson platforms) 

### Preprocessing
Using Roboflows Dataset tool I apply the following preprocessing.

    1. Auto-Orient
    2. Resize (Strectch 640x640)
    3. Auto-Adjust Contrast (Using Adaptive Equalization)
    4. Grayscale
#3 Augementation
Creating augmented version fo each image in training set by **By applying Horizontal Flip, and Vertical Flip**

## YOLOv8 Test

Is a state of the art with reasonable computational requirements, speed, and accuracy. 


In [1]:
import os
from ultralytics import YOLO
from IPython import display
from IPython.display import display, Image

# Set up home directory
HOME = os.getcwd()
print(f"Current working directory: {HOME}")

# Install required library (ensure ultralytics is installed)
!pip install ultralytics

# Clear any residual outputs
display.clear_output()

# Verify YOLO setup
import ultralytics
ultralytics.checks()

# Ensure dataset directory exists
datasets_path = os.path.join(HOME, 'data/probe_preprocessed_YOLOv8')
os.makedirs(datasets_path, exist_ok=True)

# Change directory to datasets (if needed)
os.chdir(datasets_path)
print(f"Dataset directory: {os.getcwd()}")



/home/cjs/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Current working directory: /home/cjs/Documents/Probe-Detection-Using-Deep-Learning/notebook
Defaulting to user installation because normal site-packages is not writeable


AttributeError: 'function' object has no attribute 'clear_output'

In [ ]:
# import os
# HOME = os.getcwd()
# print(HOME)

# !pip install ultralytics

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()
# from ultralytics import YOLO

# from IPython.display import display, Image

# !mkdir -p {HOME}/datasets
# %cd {HOME}/datasets

# !pip install roboflow

# # ~~~~~~~~~~~~~~~  CONNECTION TO ROBOFLOW  ~~~~~~~~~~~~~~~
# # !pip install roboflow

# # from roboflow import Roboflow
# # rf = Roboflow(api_key="VBCtrUJMVNVKifV2CuOl")
# # project = rf.workspace("julien-xkm5o").project("probe-detection-fly")
# # version = project.version(1)
# # dataset = version.download("yolov8")
                

### Train YOLOv8 model

In [ ]:
# Initialize YOLO model and train with the local dataset
model = YOLO('yolov8s.pt')  # Change 'yolov8n.pt' to the desired YOLOv8 variant
model.train(data=f"{datasets_path}/data.yaml", epochs=50, batch=16, imgsz=640)

In [ ]:

# %cd {HOME}

# !yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True
                

In [ ]:
# !ls {HOME}/runs/detect/train
!ls {HOME}/data/probe_preprocessed_YOLOv8/runs/detect/train3

In [ ]:
# %cd {HOME}
# Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)
Image(filename=f'{HOME}/data/probe_preprocessed_YOLOv8/runs/detect/train3/confusion_matrix.png', width=1000)

In [ ]:
# %cd {HOME}
# Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)
Image(filename=f'{HOME}/data/probe_preprocessed_YOLOv8/runs/detect/train3/results.png', width=1000)

In [ ]:
# %cd {HOME}
# Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=1000)
Image(filename=f'{HOME}/data/probe_preprocessed_YOLOv8/runs/detect/train3/val_batch0_pred.jpg', width=1000)

### Validate Custom Model

In [ ]:
# Validate model
model.val()

# %cd {HOME}

# !yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

### Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

Look at some example test results

In [ ]:
import glob
from IPython.display import Image, display


# Define the base path where the folders are located
base_path = f"{HOME}/runs/detect/"

    
# List all directories that start with 'predict' in the base path
subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

# Find the latest folder by modification time
latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(f'{latest_folder}/*.jpg')[:4]

# Display each image
for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n")

# Deploy Your Model to the Edge

In addition to using the Roboflow hosted API for deployment, you can use [Roboflow Inference](https://inference.roboflow.com), an open source inference solution that has powered millions of API calls in production environments. Inference works with CPU and GPU, giving you immediate access to a range of devices, from the NVIDIA Jetson to TRT-compatible devices to ARM CPU devices.

With Roboflow Inference, you can self-host and deploy your model on-device. You can deploy applications using the [Inference Docker containers](https://inference.roboflow.com/quickstart/docker/) or the pip package.

For example, to install Inference on a device with an NVIDIA GPU, we can use:

```
docker pull roboflow/roboflow-inference-server-gpu
```

Then we can run inference via HTTP:

```python
import requests

workspace_id = ""
model_id = ""
image_url = ""
confidence = 0.75
api_key = ""

infer_payload = {
    "image": {
        "type": "url",
        "value": image_url,
    },
    "confidence": confidence,
    "iou_threshold": iou_thresh,
    "api_key": api_key,
}
res = requests.post(
    f"http://localhost:9001/{workspace_id}/{model_id}",
    json=infer_object_detection_payload,
)

predictions = res.json()
```

Above, set your Roboflow workspace ID, model ID, and API key.

- [Find your workspace and model ID](https://docs.roboflow.com/api-reference/workspace-and-project-ids?ref=blog.roboflow.com)
- [Find your API key](https://docs.roboflow.com/api-reference/authentication?ref=blog.roboflow.com#retrieve-an-api-key)

Also, set the URL of an image on which you want to run inference. This can be a local file.

_To use your YOLOv5 model commercially with Inference, you will need a Roboflow Enterprise license, through which you gain a pass-through license for using YOLOv5. An enterprise license also grants you access to features like advanced device management, multi-model containers, auto-batch inference, and more._